# Init

In [1]:
%load_ext autoreload
%autoreload 2
from context import rmlib
import rmlib.rmtools as rm
from config import robotConfig

robot = rmlib.robot.Robot( robotConfig )

#Imports 
from time import sleep
import numpy as np
import rmlib
from rmlib.rmtools.assembly_trees import augment_RM

augment_RM( robot )

%store -r

####### LIBS #######################################################################################################################################

# Import Pytrees
import py_trees
from py_trees.tests import Timeout_Success
from py_trees.composites import Sequence , Selector

from assembly_trees import *


import math
from pmath import translate_pose , get_disance_between_poses , pose_components

from utils import is_matx_list
from math import radians

print( "\n##!## READY ##!##" )

Loaded: /home/nvidia/dev_rmstudio/rmlib/rmlib/rmtools Exists?: True
Loaded: /home/nvidia/dev_rmstudio/rmlib Exists?: True
Setting up robot please wait...
my_components: {'hand_config': {'class_name': 'SmartHand', 'finger_width_outer': 0.015, 'finger_length': 0.0415, 'finger_depth': 0.014, 'module_name': 'hands.smarthand', 'finger_width_inner': 0.0}, 'ft_config': {'class_name': 'OptoForce', 'ip_address': '192.168.0.3', 'module_name': 'sensors.ftsensor_optoforce'}, 'cam_config': {'class_name': 'RealSense', 'ci_cam_offset': [0, 0, 0], 'pc_cam_offset': [0, 0, 0], 'camera_model': 'd410', 'module_name': 'cameras.realsense'}, 'arm_config': {'max_linear_speed': 0.25, 'ip_address': '192.168.0.7', 'max_joint_speed': 1.05, 'default_linear_accel': 0.8, 'default_joint_accel': 0.8, 'module_name': 'arms.ur5', 'class_name': 'UR5', 'xmlrpc_port': '8003', 'max_linear_accel': 1.2, 'max_joint_accel': 1.4, 'default_joint_speed': 0.7, 'default_linear_speed': 0.1}}
active_componetns: {'ft': ['ft_config'], 'h

# Global Vars

In [2]:
DATA_RATE  = 25
SLEEP_TIME = 10.0

# Spiral Insert Process

In [3]:
from assembly_trees import _DUMMYPOSE
import py_trees

wheelPreInsertPose = wheelPostInsertPose.copy()
zApproach = 0.050
wheelPreInsertPose[2,3] += zApproach
print( "A pre-insert pose was defined." )


def get_spiral_test_tree():
    rootNode = Sequence_Recorder(  memory = 0 , dataKey = "test_DATA" , flagKey = "test_FLAG" , 
                                   outFileGenFunc = get_XML_outfile_namer( pathPrefix = "output/" , 
                                                                           namePrefix = "GEAR-SPR_" )  )

    recordBehav = Record_Classify( recordFlagKey = "test_FLAG" , 
                                   dataKey       = "test_DATA" , 
                                   recordHz = 50, ctrl = robot )


    insertTree = Sequence( memory = 1 )
    
    waitBehav = Timeout_Success( timeout = 0.20 )

    goBehav = Jog_Safe( wheelGraspPose , 
                        zSAFE = 0.100 ,  
                        hover = 1 ,  
                        ctrl  = robot  )

    graspBehav = Grasp_at_Pose( wheelGraspPose, 0.026 ,
                                zApproach=0.050 , zClose=0.0 , wdthNarrow = 0.050 , maxIter=5, zFree=0.100 , 
                                ctrl = robot )

    jogBehav = Jog_Safe( wheelPreInsertPose, 
                         zSAFE = 0.100 ,  
                         hover = 0 ,  
                         ctrl  = robot  )

    relaxedInsert = Selector( name = "Spiral_ins_ALWAYSDROP" , memory = 1 )
    
    stepFactor = 2.0
    
    # 1.8 # Lateral torque caused it to bind
    insertBehav = Spiral_Insert( 
        wheelPreInsertPose ,
        touch_force = 2.0 , drop_force = 1.0 , insert_force = 9.0 , max_movement = zApproach + 0.030 , 
        lateralStopTorque = 0.475 , 
        pushbackF = 20.0 , spiralSpeed = 0.001 , descendSpeed = 0.00625 , biasWrist = 1 ,
        degrees_to_step = 3.25*2.0 , 
        maxAngle = 100 * 360 , startRadius = 0.00005 , 
        stepSize = 0.000025*1.75 , 
        max_radius = 0.010 ,
        posnMargin = 0.003 , orntMargin = 2.0/180 , 
        reliefStep_m = 0.0005 , reliefMargin_Nm = 0.050 , reliefN = 15 , 
        finalInsertHandZ = wheelPostInsertZ , 
        suppressDrop = 1 , suppressLateral = 0 , 
        recordFlagKey = "test_FLAG" , 
        ctrl = robot,
        chaseMode = 1, chaseStep = 0.00010 , N_chaseMax = 30 
    )                    
    
    dropBehav = Set_Fingers( openState = 0.060 , ctrl = robot ) 
    
    relaxedInsert.add_children( [ insertBehav , py_trees.decorators.SuccessIsFailure( dropBehav ) ] )
    
    cntrBehav = Jog_Safe( wheelPreInsertPose , #wheelPreOffsetYPose,  # wheelPreInsertPose , 
                         zSAFE = 0.100 ,  
                         hover = 0 ,  
                         ctrl  = robot  )
    
    insertTree.add_children( [ goBehav , graspBehav , jogBehav , relaxedInsert, dropBehav ] ) #, cntrBehav , checkBehav ] )

    rootNode.add_children( [ recordBehav , insertTree ] ) #, waitBehav ] )
    
    return rootNode

print( "Spiral BT Created!" )

A pre-insert pose was defined.
Spiral BT Created!


# Spiral Insert: Recording Pipeline

In [4]:
N        =  1
divSleep = 10
t_sleep  = 60
j        =  1

from time import sleep

for i in range( 1 , N+1 ):
    
    try:
        
        print( "Iteration" , i , "of" , N )
        
        # 6. Sleep for human reset
        if i > 1:
            sleep( divSleep )

        # 3. Release
        robot.hand.release()
        
        # 1. Reset tree
        rootNode = get_spiral_test_tree()
        
        # 2. Run tree and generate recording
        run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , 
                           breakOnFailure = 1 , breakOnSuccess = 1 , Nverb = 0 )
        
        # 3. Release
        robot.hand.set_finger_width( 0.050 ) 
        
        # 4. Deactivate motors
        robot.hand.deactivate_motors()
        
        # 5. Move to safe position
        robot.arm.move( safeCenterPose )

    except KeyboardInterrupt:
        print( "Pipeline was STOPPED by user at iteration" , i+1 )
        break
        

Iteration 1 of 1
Root node Sequence_Recorder failed!


Simulation completed! with status rootNode.status 




# Tilt Insert: Shaft Recording

In [3]:
def get_shaft_insert_tree():
    rootNode = Sequence_Recorder(  memory = 0 , dataKey = "test_DATA" , flagKey = "test_FLAG" , 
                                   outFileGenFunc = get_XML_outfile_namer( pathPrefix = "output/" , namePrefix = "SHAFT-TLT_" )  )

    recordBehav = Record_Classify( recordFlagKey = "test_FLAG" , dataKey = "test_DATA" , recordHz = DATA_RATE , ctrl = robot )


    insertTree = Sequence( memory = 1 )

    goBehav = Jog_Safe( shaftGraspPose , 
                        zSAFE = 0.100 ,  
                        hover = 1 ,  
                        ctrl  = robot  )

    closeBehav = Set_Fingers( openState = 0.040 , ctrl = robot )   
    
    graspBehav = Grasp_at_Pose( shaftGraspPose, 0.015 ,
                                zApproach=0.050 , zClose=0.0 , wdthNarrow = 0.030 , maxIter=5, zFree=0.100 , 
                                ctrl = robot )

    jogBehav = Jog_Safe( shaftPreInsertPose , 
                        zSAFE = 0.100 ,  
                        hover = 0 ,  
                        ctrl  = robot  )


    tiltBehav = Tilt_Insert( shaftPreInsertPose ,
                              tilt_angle_deg = 7.25 , part_offset = 0.016 , dia = 0.015 , 
                              touch_force = 0.6 , insert_force = 1.5 , max_movement = 0.045 , biasWrist = 1 , offset_dir = np.array([1.0,0.0,0.0]), 
                              posnMargin = 0.003 , orntMargin = 2.0/180 , finalInsertHandZ = fatPegFullInsertZ , recordFlagKey = "test_FLAG" , 
                              dropSpeed = 0.0625 , ctrl = robot )

    dropBehav = Set_Fingers( openState = 1.0 , ctrl = robot )    

    insertTree.add_children( [ goBehav , closeBehav , graspBehav , jogBehav , tiltBehav , dropBehav ] )

    rootNode.add_children( [ recordBehav , insertTree ] )
    
    return rootNode

# Tilt Insert Pipeline

In [4]:
N        =  3
divSleep = 10
t_sleep  = 90
j        =  1

N_sleep = int( N / divSleep )

from time import sleep

for i in range( 1 , N+1 ):
    
    try:
        
        print( "Iteration" , i , "of" , N )
        
        # 6. Sleep for human reset
        if i > 1:
            sleep( divSleep )
        
        # 1. Reset tree
        rootNode = get_shaft_insert_tree()
        
        # 2. Run tree and generate recording
        run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 , Nverb = 0 )
        
        # 3. Release
        robot.hand.release()
        
        # 4. Deactivate motors
        robot.hand.deactivate_motors()
        
        # 5. Move to safe position
        robot.arm.move( safeCenterPose )
        
        # TODO: Is this rest the same as a shutdown
        if i%divSleep == 0:
            print( "Sleep" , j , "of" , N_sleep , ", Sleep for" , t_sleep , "seconds..." )
            sleep( t_sleep )
            j += 1

    except KeyboardInterrupt:
        print( "Pipeline was STOPPED by user at iteration" , i+1 )
        break

Iteration 1 of 3
Root node Sequence_Recorder failed!


Simulation completed! with status rootNode.status 


Iteration 2 of 3
Root node Sequence_Recorder failed!


Simulation completed! with status rootNode.status 


Iteration 3 of 3
Root node Sequence_Recorder failed!


Simulation completed! with status rootNode.status 




# Testing

In [ ]:
N = 10
from time import sleep

for i in range( N ):
    
    try:
        
        print( "Iteration" , i+1 , "of" , N )
            
#         robot.ft.set_stream_Hz200()
        
#         print( robot.ft.get_wrist_force() )
        
        robot.hand.release()
        
        robot.arm.move( wheelPreInsertPose )
        
        # 1. Reset tree
        rootNode = Tamp_Alternating( zGoal = 0.0472 , zMargin = 0.005 , tap_force = 5.0 , plungeDistance = 0.100 , tapWidth = 0.020 ,
                                   twist_deg = 90 , doubleSided = 0 , retract_m = 0.040 , 
                                   descendSpeed = 0.0125 , Ntries = 1 , TzStop = 1.8 , ctrl = robot )
        
        # 2. Run tree and generate recording
        run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 , Nverb = 0 )
        
        # 3. Release
        robot.hand.release()
        
        # 4. Deactivate motors
        robot.hand.deactivate_motors()
        
        # 5. Move to safe position
#         robot.arm.move( safeCenterPose )

        # 6. Sleep for human reset
        sleep( 5 )
        
        
    except KeyboardInterrupt:
        print( "Pipeline was STOPPED by user at iteration" , i+1 )
        break

# Common Functions

In [21]:
robot.arm.align_tcp( robot.arm )

True

In [19]:
robot.hand.grip()

True

In [ ]:
robot.hand.release()

In [18]:
robot.hand.set_finger_width( 0.040 ) 

True

In [ ]:
robot.arm.move( wheelGraspPose )

In [ ]:
print( robot.arm.get_tcp_pose() )
print( robot.arm.get_tcp_pose()[2,3] )
print( baseTouchPoseZ )

In [ ]:
robot.ft.get_wrist_force()

In [ ]:
robot.hand.deactivate_motors()

In [ ]:
robot.hand.zero_fingers()

# Setup

### Gear Poses

In [20]:
# wheelGraspPose = robot.arm.get_tcp_pose()
# %store wheelGraspPose

Stored 'wheelGraspPose' (ndarray)


In [28]:
# wheelPostInsertPose = robot.arm.get_tcp_pose()
# wheelPostInsertZ = wheelPostInsertPose[2,3]
# %store wheelPostInsertPose
# %store wheelPostInsertZ
# wheelPreInsertPose = wheelPostInsertPose.copy()
# zApproach = 0.070
# wheelPreInsertPose[2,3] += zApproach
# %store wheelPreInsertPose

Stored 'wheelPostInsertPose' (ndarray)
Stored 'wheelPostInsertZ' (float64)


In [ ]:
# wheelPreOffsetYPose = robot.arm.get_tcp_pose()
# %store wheelPreOffsetYPose

In [ ]:
# wheelPreOffsetXPose = robot.arm.get_tcp_pose()
# %store wheelPreOffsetXPose

### Shaft Poses

In [13]:
# shaftGraspPose = robot.arm.get_tcp_pose()
# %store shaftGraspPose

Stored 'shaftGraspPose' (ndarray)


In [14]:
# shaftPreInsertPose = robot.arm.get_tcp_pose()
# %store shaftPreInsertPose

Stored 'shaftPreInsertPose' (ndarray)


### General Poses

In [6]:
# safeCenterPose = robot.arm.get_tcp_pose()
# %store safeCenterPose

Stored 'safeCenterPose' (ndarray)


In [7]:
# baseTouchPose  = robot.arm.get_tcp_pose()
# baseTouchPoseZ = baseTouchPose[2,3]
# %store baseTouchPose
# %store baseTouchPoseZ

Stored 'baseTouchPose' (ndarray)
Stored 'baseTouchPoseZ' (float64)


# TEST: Relieve Wrist Torque
1. Put the wrist in a slighly torqued place

In [22]:
py_trees.logging.level = py_trees.logging.Level.INFO

rootNode =  Minimize_Wrist_Torque_XY( numTrials = 10 , marginT=0.10 , moveStep=0.002  , ctrl = robot )




In [26]:
print( "FT before the bias:" , robot.ft.get_wrist_force() )
robot.ft.bias_wrist_force()
sleep(1)
print( "FT after  the bias:" , robot.ft.get_wrist_force() )

FT before the bias: [0.04999995231628418, 0.0, 0.199981689453125, 0.003000020980834961, -0.0015000104904174805, 0.0]
FT after  the bias: [0.0, 0.0, 0.0, 0.0004999637603759766, -0.0004999637603759766, -0.0004999935626983643]


APPLY TORQUE HERE

In [27]:
print( "FT before the behavior:" , robot.ft.get_wrist_force() )

run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 )

print( "FT After the behavior:" , robot.ft.get_wrist_force() )

FT before the behavior: [0.6000001430511475, 1.399999976158142, 1.04998779296875, -0.24875009059906006, 0.04674994945526123, -0.007749989628791809]
Root node Minimize_Wrist_Torque_XY succeeded!


Simulation completed! with status rootNode.status 


FT After the behavior: [0.04375004768371582, 0.0, 0.240631103515625, 0.0010156631469726562, 0.00267183780670166, -0.000734373927116394]
